In [ ]:
from skimage.measure import label, regionprops
from skimage.transform import resize

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from glob import glob
import re

from multiprocessing.pool import ThreadPool

import pydicom

In [ ]:
from model.MultiResUNet3D import MultiResUnet3D
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.
D:\Programas\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Programas\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Programas\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Programas\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Fu

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
set_session(tf.Session(config=config))

In [ ]:
root_mask_path = "Y:/LIDC-IDRI_final_masks/LIDC-IDRI-{}.npz"
save_path = './binary/'
n_classes = 2
chanels = 1
mask_size = np.array([64, 64, 64])
patch_size = np.array([10, 20, 20])

In [ ]:
def get_coods_matrix(start, stop):

    z = np.arange(start[0], stop[0], 64)
    z[len(z) - 1] = stop[0] - 64

    y = np.arange(start[1], stop[1], 64)
    y[len(y) - 1] = stop[1] - 64

    x = np.arange(start[2], stop[2], 64)
    x[len(x) - 1] = stop[2] - 64

    indexes = np.array(np.meshgrid(z, y, x))
    indexes = indexes.reshape(3, -1).T

    return indexes

def index_to_slice(start, thickness):
    slices = []
    for i in start:
        t = slice(i, i + thickness)
        slices.append(t)
    return slices

def get_dicoms_from_folder(folder_name, jobs=30):
    try:
        list_url = glob(folder_name)
        dcms = list(ThreadPool(jobs).imap_unordered(pydicom.dcmread, list_url))
    except Exception as e:
        raise Exception('error al paralelizar la descarga ' + str(e))
    return dcms

def get_vol_from_dcms(dcms):
    dcms.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    try:
        vol = []
        spacing = np.array(dcms[0].PixelSpacing).astype(np.float)
        thickness = float(abs(dcms[0].ImagePositionPatient[2] - dcms[1].ImagePositionPatient[2]))
        spacing = np.concatenate([[thickness], spacing])

        for dcm in dcms:
            intercept = int(dcm.RescaleIntercept)
            slope = int(dcm.RescaleSlope)
            img_2d = dcm.pixel_array.copy().astype(np.int16)
            if slope != 1:
                img_2d *= slope

            img_2d += intercept
            vol.append(img_2d)

        vol = np.array(vol, dtype=np.int16)
    except Exception as e:
        print(str(e))
        raise Exception('error extraer el volumen del dicom ' + str(e))
    return vol, spacing

def make_predictions_fom_vol(vol, indexes):
    try:
        thickness = 64
        pred = np.zeros((*vol.shape, 2))
        for start in indexes:
            slices = index_to_slice(start, thickness)
            cube = vol[slices]
            temp = model_nodules_ct.predict(cube[np.newaxis, ..., np.newaxis]) # *cube.shape, 1
            pred[slices] = temp[0]
    except Exception as e:
        print(str(e))
        raise('error al hacer una de las predicciones de los nodulos' + str(e))
    return pred

def jaccard(a, b):
    a = np.asarray(a).astype(np.bool)
    b = np.asarray(b).astype(np.bool)

    if a.shape != b.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    intersection = np.logical_and(a, b)
    union = np.logical_or(a, b)
    return intersection.sum() / float(union.sum())

def normalize(npzarray):
    maxHU = 400.
    minHU = -1000.
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray = np.clip(npzarray, 0, 1)
    return npzarray

In [ ]:
model_nodules_ct = MultiResUnet3D(mask_size[0], mask_size[1], mask_size[2], chanels, n_classes)
model_nodules_ct.load_weights('./weights/weights.08.hdf5')

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
train, test = np.load('splited.npy', allow_pickle=True)

In [ ]:
paths = [*train, *test]

In [ ]:
count = 0
all_data = []

In [ ]:
for path in paths:
    dcms = get_dicoms_from_folder(path+'/*.dcm')
    vol, spacing = get_vol_from_dcms(dcms)
    case = re.search('LIDC-IDRI-(.+?)/', path).group(1)
    
    mask_path = root_mask_path.format(case)
    file = np.load(mask_path)
    mask = file['mask']
    file.close()
    
    if mask.shape != vol.shape:
        continue
    
    start = (0, 0, 0)
    stop = vol.shape
    indexes = get_coods_matrix(start, stop)
    
    pred = make_predictions_fom_vol(normalize(vol), indexes).argmax(-1)
    
    labeled_pred = label(pred)
    labeled_mask = label(mask)
    
    props_pred = regionprops(labeled_pred)
    
    n_pred = len(props_pred)
    n_true = int(labeled_mask.max())
    
    for prop in props_pred:
        min_slice, min_row, min_col, max_slice, max_row, max_col = prop.bbox
        
        true_mask = labeled_mask[min_slice:max_slice, min_row:max_row, min_col:max_col].copy()
        pred_mask = labeled_pred[min_slice:max_slice, min_row:max_row, min_col:max_col].copy()
        patch = vol[min_slice:max_slice, min_row:max_row, min_col:max_col]
        
        vals, counts = np.unique(true_mask[pred_mask == prop.label], return_counts=True)
        
        if len(vals) == 1 and vals[0] == 0:
            tag = np.array([0], dtype=np.uint8)
            j_index = -1
        else:
            counts = counts[vals != 0]
            vals = vals[vals != 0]
            campare_label = vals[vals == vals.max()][0]
            
            true_mask = true_mask == campare_label
            pred_mask = pred_mask == prop.label

            j_index = jaccard(labeled_mask == campare_label, labeled_pred == prop.label)

            tag = np.array([j_index > 0.5], dtype=np.uint8)
            
        path_new = './binary/{}.npz'.format(count)
        data = {'tag': int(tag[0]), 'path': path_new, 'id': count, 'jaccard_index': float(j_index), 'n_pred': n_pred, 'n_true': n_true, 'study_path': path}
        
        all_data.append(data)
        np.savez_compressed(path_new, patch = patch, tag = tag, true_mask=true_mask, pred_mask=pred_mask)
        
        count+=1

D:\Programas\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
D:\Programas\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [ ]:
data_df = pd.DataFrame(all_data)

In [ ]:
data_df[data_df.jaccard_index != -1]

In [ ]:
import subprocess

# Apagar
subprocess.call("shutdown -s")